В файле предоставленна информация о клиентах риэлторской компании США за один месяц. Вам необходимо как специалисту по Дата Сайнс сделать следующее:
- Используя данные о выборке, постройте типичный портрет целевого клиента компании:
    - пол
    - возраст
    - какой тип квартиры он приобретает
    - какова площадь в кв. м
    - каков средний чек
    - цель покупки
    - источник лида (погуглите что такое лид в маркетинге, за это отвечает колонка Source).
Таким образом после вашего анализа, необходимо получить небольшое описание нашего целевого клиента с соответствующими визуализациями.

In [49]:
#импортируем все самое нужное
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
# Using plotly + cufflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

In [50]:
#загрузим датасет 
df = pd.read_csv(r'clients.csv')
df.head()

,ID,Building,Year of sale,Month of sale,Type of property,Property number,Area (ft.),Price,Status,Customer ID,...,Y,M,D,Gender,Country,State,Purpose,Deal satisfaction,Mortgage,Source
0,1030,1,2005,11,Apartment,30,743.09,"$ 246,172.68",Sold,C0028,...,1986.0,6.0,21.0,F,USA,California,Home,5,No,Website
1,1029,1,2005,10,Apartment,29,756.21,"$ 246,331.90",Sold,C0027,...,1983.0,2.0,24.0,F,USA,California,Home,5,No,Website
2,2002,2,2007,7,Apartment,2,587.28,"$ 209,280.91",Sold,C0112,...,1985.0,12.0,27.0,M,USA,California,Home,1,Yes,Client
3,2031,2,2007,12,Apartment,31,1604.75,"$ 452,667.01",Sold,C0160,...,1985.0,12.0,27.0,M,USA,California,Investment,3,Yes,Website
4,1049,1,2004,11,Apartment,49,1375.45,"$ 467,083.31",Sold,C0014,...,1979.0,5.0,15.0,F,USA,California,Home,4,No,Agency


Посмотрим, из чего же состоит наш датасет, и как мы можем собрать усредненный профиль нашего клиента. Какие есть показатели.

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       195 non-null    int64  
 1   Building                 195 non-null    int64  
 2   Year of sale             195 non-null    int64  
 3   Month of sale            195 non-null    int64  
 4   Type of property         195 non-null    object 
 5   Property number          195 non-null    int64  
 6   Area (ft.)               195 non-null    float64
 7   Price                    195 non-null    object 
 8   Status                   195 non-null    object 
 9   Customer ID              195 non-null    object 
 10  Entity                   195 non-null    object 
 11  Name                     195 non-null    object 
 12  Surname                  195 non-null    object 
 13  Age at time of purchase  178 non-null    float64
 14  Age Interval             1

In [52]:
df.isnull().sum()[df.isnull().sum()!=0]

Age at time of purchase    17
Age Interval               17
Y                          17
M                          17
D                          17
Gender                     17
State                      14
dtype: int64

Видим, что в нашем датасете есть пропущенные значения, но ничего делать с этим пока не будем, т.к. данные на данном этапе нам нужны только для анализа, а не для последующего использования.
Однако есть колонки, по которым некорректно определился тип, это исправим.

In [53]:
df['Price'].replace(inplace=True, regex=True, to_replace=['\$ ', ','], value=['',''])
df.Price = df.Price.astype('float64')
df.head()

,ID,Building,Year of sale,Month of sale,Type of property,Property number,Area (ft.),Price,Status,Customer ID,...,Y,M,D,Gender,Country,State,Purpose,Deal satisfaction,Mortgage,Source
0,1030,1,2005,11,Apartment,30,743.09,246172.68,Sold,C0028,...,1986.0,6.0,21.0,F,USA,California,Home,5,No,Website
1,1029,1,2005,10,Apartment,29,756.21,246331.90,Sold,C0027,...,1983.0,2.0,24.0,F,USA,California,Home,5,No,Website
2,2002,2,2007,7,Apartment,2,587.28,209280.91,Sold,C0112,...,1985.0,12.0,27.0,M,USA,California,Home,1,Yes,Client
3,2031,2,2007,12,Apartment,31,1604.75,452667.01,Sold,C0160,...,1985.0,12.0,27.0,M,USA,California,Investment,3,Yes,Website
4,1049,1,2004,11,Apartment,49,1375.45,467083.31,Sold,C0014,...,1979.0,5.0,15.0,F,USA,California,Home,4,No,Agency


In [54]:
df.Price.dtype

dtype('float64')

И предусмотрительно заменим метку гендера на тот, который потом удобно будет выводить на графики.

In [55]:
df['Gender'].replace(inplace=True, regex=True, to_replace=['F', 'M'], value=['Female','Male'])
df.head()

,ID,Building,Year of sale,Month of sale,Type of property,Property number,Area (ft.),Price,Status,Customer ID,...,Y,M,D,Gender,Country,State,Purpose,Deal satisfaction,Mortgage,Source
0,1030,1,2005,11,Apartment,30,743.09,246172.68,Sold,C0028,...,1986.0,6.0,21.0,Female,USA,California,Home,5,No,Website
1,1029,1,2005,10,Apartment,29,756.21,246331.90,Sold,C0027,...,1983.0,2.0,24.0,Female,USA,California,Home,5,No,Website
2,2002,2,2007,7,Apartment,2,587.28,209280.91,Sold,C0112,...,1985.0,12.0,27.0,Male,USA,California,Home,1,Yes,Client
3,2031,2,2007,12,Apartment,31,1604.75,452667.01,Sold,C0160,...,1985.0,12.0,27.0,Male,USA,California,Investment,3,Yes,Website
4,1049,1,2004,11,Apartment,49,1375.45,467083.31,Sold,C0014,...,1979.0,5.0,15.0,Female,USA,California,Home,4,No,Agency


Также площадь помещения в датасете указана в футах, однако нам удобнее оперировать м2. Пересчитаем. 1 фут2 = 10.757 м2 

In [56]:
df[r'Area (ft.)'] = df[r'Area (ft.)']/10.67
df.rename(columns={r'Area (ft.)': r'Area (m2.)'}, inplace=True)
df.head()

,ID,Building,Year of sale,Month of sale,Type of property,Property number,Area (m2.),Price,Status,Customer ID,...,Y,M,D,Gender,Country,State,Purpose,Deal satisfaction,Mortgage,Source
0,1030,1,2005,11,Apartment,30,69.642924,246172.68,Sold,C0028,...,1986.0,6.0,21.0,Female,USA,California,Home,5,No,Website
1,1029,1,2005,10,Apartment,29,70.872540,246331.90,Sold,C0027,...,1983.0,2.0,24.0,Female,USA,California,Home,5,No,Website
2,2002,2,2007,7,Apartment,2,55.040300,209280.91,Sold,C0112,...,1985.0,12.0,27.0,Male,USA,California,Home,1,Yes,Client
3,2031,2,2007,12,Apartment,31,150.398313,452667.01,Sold,C0160,...,1985.0,12.0,27.0,Male,USA,California,Investment,3,Yes,Website
4,1049,1,2004,11,Apartment,49,128.908154,467083.31,Sold,C0014,...,1979.0,5.0,15.0,Female,USA,California,Home,4,No,Agency


Посмотрим как распределялись доходы компании по годам. На графике заметно выделяется 2007 год по общей сумме продаж. Хороший видимо был год.

In [57]:
df[['Year of sale', 'Price']].groupby(by=['Year of sale']).sum().iplot(kind='bar', 
                                                                      xTitle='Years', 
                                                                      yTitle='Sales, M$',
                                                                      title='Sales');

Однако по общим суммам не совсем корректно делать выводы по стоимости недвижимости на рынке, посмотрим на динамику средней цены за м2. 

In [58]:
df['Price per m2'] = df['Price']/df[r'Area (m2.)']
df[['Year of sale', 'Price per m2']].groupby(by=['Year of sale']).mean().iplot(mode='lines+markers+text', 
                                                                               xTitle='Year',
                                                                               yTitle='$/m2',
                                                                               title='Price per m2');

Видим, что в 2010 году тренд сильно подскочил вверх. Посмотрим, что за продажи были в этом году.

Оказывается это одна сделка по нетипично высокой цене. Для расчета средней стоимости недвижимости в этом случае предлагаю ее не учитывать. Но на практике конечно сначала хотелось бы больше узнать про рынок и принимать решение об учете информации исходя из текущих трендов.

In [59]:
df[df['Year of sale']==2010]

,ID,Building,Year of sale,Month of sale,Type of property,Property number,Area (m2.),Price,Status,Customer ID,...,M,D,Gender,Country,State,Purpose,Deal satisfaction,Mortgage,Source,Price per m2
116,5030,5,2010,5,Apartment,30,99.149016,357538.2,Sold,C0094,...,4.0,20.0,Male,USA,Nevada,Home,3,Yes,Website,3606.069073


Построим для наглядности круговую диаграму распределения покупателей по полу. Существенно преобладают мужчины.

In [60]:
gender = df['Gender'].value_counts()
fig = px.pie(gender, values='Gender', names=gender.index, color_discrete_sequence=px.colors.qualitative.Pastel2)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

Построим диаграмму продаж по годам и странам. Очевидно во всех периодах лидирует США.

In [61]:
fig = px.bar(df[['Price', 'Year of sale', 'Country']].groupby(by=['Year of sale', 'Country'], as_index=False).sum(), 
             x="Year of sale", 
             y="Price", 
             color='Country', 
             text_auto=True)
fig.show()

Построим график распределения покупателей по возрасту. Видим, что чаще всего покупка недвижимости совершается по этим данным в возрасте 48 лет.

In [62]:
df['Age at time of purchase'].value_counts().iplot(kind='bar')

В целом видим, что большинство клиентов по данному датасету удовлетворены своей покупкой и обслуживанием агентсва, большинство оценок 4-5. 

In [136]:
df['Deal satisfaction'].value_counts().iplot(kind='bar', barmode='stack', orientation='h')

По типу собственности абсолютное преимущество конечно у аппартаментов.

In [137]:
gender = df['Type of property'].value_counts()
fig = px.pie(gender, values='Type of property', names=gender.index, color_discrete_sequence=px.colors.qualitative.Pastel2)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

Посчитаем некоторые усредненные числовые значения, такие как средняя цена покупки и площадь помещения в м2.

In [138]:
df[['Price', r'Area (m2.)', 'Type of property']].groupby(by=['Type of property'], as_index=False).mean()

,Type of property,Price,Area (m2.)
0,Apartment,270038.541755,84.499397
1,Office,253213.361429,79.952872


Для всех остальных категориальных признаков построим диаграммы, отражающие количество записей в датасете для каждой категории. Сделаем все в одном графике.

In [156]:
cat_features = ['Entity', 'Purpose', 'Mortgage', 'Source']

In [140]:
from plotly.subplots import make_subplots
fig = make_subplots(rows=len(cat_features), cols=1)

for feature in cat_features:
    fig.add_bar(df[feature].value_counts(normalize=True).iplot(kind='bar', orientation='h', title=feature))

###### По всему проведенному выше анализу можно сделать вывод, что наш клиент:
1. Мужчина (60.7%)
2. Проживает в США.
3. Совершает покупку недвижимости в 48 лет.
4. Покупает квартиру (96.4%)
5. Площадью 84.5 м2
6. Стоимостью 270 038.54 долл (или 3 198.18 долл/м2)
7. Цель его покупки - покупка жилья (61.02%) 
8. Покупает как физ лицо (entity - individual, 91.28%)
9. За собственные денежные средства (68.72%).
10. Был привлечен в текущее агенство по недвижимости с поощью сайта (61.02%)